# Generate platemap visualizations


## Import libraries

In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(platetools))

Warning message:
“package ‘ggplot2’ was built under R version 4.2.3”
Warning message:
“package ‘platetools’ was built under R version 4.2.3”


## Set paths

In [2]:
platemap_files <- list.files(pattern = "_platemap\\.csv$", full.names = TRUE)
print(platemap_files)

output_fig_dir <- file.path("platemap_figures")
if (!dir.exists(output_fig_dir)) {
    dir.create(output_fig_dir, recursive = TRUE)
}
platemap_suffix <- "_platemap_figure.png"

# Define output figure paths
output_platemap_files <- list()
for (platemap_file in platemap_files) {
    # Extract plate name and remove suffix 
    plate <- basename(platemap_file)
    plate <- stringr::str_remove(plate, "_platemap.csv") 
    
    output_platemap_files[[plate]] <- file.path(output_fig_dir, paste0(plate, platemap_suffix))
}

print(output_platemap_files)

[1] "./NF0014_platemap.csv"
$NF0014
[1] "platemap_figures/NF0014_platemap_figure.png"



## Load in platemap file

In [3]:
# Load in all platemap CSV files
platemap_dfs <- list()
for (plate in names(output_platemap_files)) {
    # Find the platemap file associated with the plate
    platemap_file <- platemap_files[stringr::str_detect(platemap_files, plate)]
    
    # Load in the platemap data
    df <- readr::read_csv(
        platemap_file,
        col_types = readr::cols(.default = "c")
    )
    
    # Update 'Dose' column
    df <- df %>%
        mutate(
            dose = case_when(
                treatment == "DMSO" ~ paste0(dose, "%"),
                TRUE ~ paste0(dose, " uM")
            )
        )
    
    # Store the updated data frame
    platemap_dfs[[plate]] <- df 
}

# Print the head of each data frame in the list
for (plate in names(platemap_dfs)) {
    cat("\nHead of plate:", plate, "\n")
    print(head(platemap_dfs[[plate]]))
}


Head of plate: NF0014 
# A tibble: 6 × 5
  WellRow WellCol well_position treatment dose 
  <chr>   <chr>   <chr>         <chr>     <chr>
1 C       2       C2            Staurosporine    10 uM
2 D       2       D2            Digoxin   1 uM 
3 E       2       E2            Digoxin   1 uM 
4 F       2       F2            Onalespib 1 uM 
5 G       2       G2            Staurosporine    10 uM
6 C       3       C3            Onalespib 1 uM 


## Create example platemaps and save figures

In [10]:
# Color-blind friendly and most distinct color palette
okabe_ito <- c(
    "#E69F00", "#56B4E9", "#009E73", "#F0E442", "#8DA0CB", 
    "#D55E00", "#CC79A7", "#999999", "#A6CEE3", "#1F78B4", 
    "#B2DF8A", "#33A02C", "#FB9A99", "#E31A1C", "#FDBF6F", 
    "#FF7F00", "#CAB2D6"
)

for (plate in names(platemap_dfs)) {
    output_file <- output_platemap_files[[plate]]
    platemap <- platetools::raw_map(
        data = platemap_dfs[[plate]]$treatment,
        well = platemap_dfs[[plate]]$well_position,
        plate = 96,
        size = 10
    ) +
        ggtitle(paste("Platemap layout for plate", plate)) +
        theme(
            plot.title = element_text(size = 10, face = "bold"),
            legend.position = "right",
            legend.box = "vertical",
            legend.spacing.y = unit(0.5, "cm"),
            legend.margin = margin(t = 0, b = 5, unit = "pt"),
            legend.text = element_text(size = 8),
            legend.title = element_text(size = 9)
        ) +
        geom_point(aes(shape = platemap_dfs[[plate]]$dose)) +
        scale_shape_discrete(name = "Dose") +
        scale_fill_manual(values = okabe_ito, name = "Treatment") +  # Use Okabe-Ito palette
        guides(
            shape = guide_legend(order = 2, nrow = 1, override.aes = list(size = 3)),
            fill = guide_legend(order = 1, ncol = 2, override.aes = list(size = 3))
        )

    ggsave(output_file, platemap, device = 'png', dpi = 500, height = 4, width = 8)
}
